In [4]:
%cd /work/pi_wqd_umass_edu/pyxsim

/work/pi_wqd_umass_edu/pyxsim


In [5]:
import yt
import pyxsim
import re
import struct
import sys
import warnings
from io import open
import h5py
import astropy
from matplotlib.colors import LogNorm
from astropy.wcs import WCS
from astropy.io import fits
from scipy.ndimage import rotate
from astropy import wcs
import astropy.units as u
from pprint import pprint

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pyfits
import matplotlib
import soxs
matplotlib.rc('font', size = 18)

In [72]:
from yt.units import pc, mp

In [32]:
M_NSC = 6.1e7
q = .73
a = 147.6
a_0 = 147.6
y = .71
R = 1

In [33]:
def density_function(M_NSC, q, a_0, a,y):
    density = ((3-y)*M_NSC / (4 * np.pi * q)) * (a_0 / a**y*(a+a_0)**(4-y))
    return density

In [34]:
grid_size = (128,128,128)
x = np.linspace(-1, 1, grid_size[0])
y = np.linspace(-1, 1, grid_size[1])
z = np.linspace(-1, 1, grid_size[2])
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
r = np.sqrt(x**2 + y**2 + z**2)

In [35]:
dens = np.zeros(grid_size)
dens[r <= R] = density
dens[r > R] = 0.0

In [41]:
data = {}
data["density"] = (dens)
data["velocity_x"] = (np.zeros(grid_size))
data["velocity_y"] = (np.zeros(grid_size))
data["velocity_z"] = (np.zeros(grid_size))

domain_dimensions = grid_size
bbox = np.array([[-1, 1], [-1, 1], [-1, 1]])
# Load the grid into yt
ds = yt.load_uniform_grid(data, domain_dimensions, bbox=bbox,default_species_fields='ionized')

yt : [INFO     ] 2024-08-05 16:31:29,034 Parameters: current_time              = 0.0
yt : [INFO     ] 2024-08-05 16:31:29,035 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2024-08-05 16:31:29,035 Parameters: domain_left_edge          = [-1. -1. -1.]
yt : [INFO     ] 2024-08-05 16:31:29,036 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-08-05 16:31:29,036 Parameters: cosmological_simulation   = 0


In [26]:
"""density = density_function(M_NSC, q, a_0, a,y)

# Create a dictionary for the data
data = {}
data["density"] = density
#data["temperature"] = (temp, "K")
data["velocity_x"] = np.zeros(grid_size)
data["velocity_y"] = np.zeros(grid_size)
data["velocity_z"] = np.zeros(grid_size)

# Define the domain dimensions
domain_dimensions = grid_size
bbox = np.array([[-1, 1], [-1, 1], [-1, 1]])
# Load the grid into yt
ds = yt.load_uniform_grid(data, domain_dimensions, bbox=bbox,default_species_fields="ionized")

yt : [INFO     ] 2024-08-05 16:01:29,450 Parameters: current_time              = 0.0
yt : [INFO     ] 2024-08-05 16:01:29,451 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2024-08-05 16:01:29,451 Parameters: domain_left_edge          = [-1. -1. -1.]
yt : [INFO     ] 2024-08-05 16:01:29,452 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-08-05 16:01:29,453 Parameters: cosmological_simulation   = 0


TypeError: '<' not supported between instances of 'tuple' and 'str'

In [80]:
t = 10e7
r = 8.18 "Kpc"
r_0 = 1 "pc"
a = .5

SyntaxError: invalid syntax (261374430.py, line 2)

In [79]:
def temp(r,r_0,t):
    temp = t(r_0/r)**a
ds.add_field(
    ("gas", "temperature"),
    function=
    sampling_type="local",
    units="K",
force_override = True)

TypeError: 0.025 is not a callable object

In [75]:
source_model = pyxsim.CIESourceModel("apec", 1.0, 10.0, 6000, Zmet = 3)

pyxsim : [INFO     ] 2024-08-05 20:21:56,060 kT_min = 0.025 keV
pyxsim : [INFO     ] 2024-08-05 20:21:56,061 kT_max = 64 keV


In [76]:
exp_time = (2000.0, "ks") #shorter exposure time because my computer has no memory.
area = (1500.0, "cm**2") 
redshift = 0 # its zero at first because it is local but I believe later I will be adding redshift.
dist = (8.18, "kpc")

In [77]:
sp = ds.sphere("c", (1.0, "pc"))

In [78]:
n_photons, n_cells = pyxsim.make_photons("bulge_test", sp, redshift,area, exp_time, source_model, dist = dist)

pyxsim : [INFO     ] 2024-08-05 20:22:00,730 Observing local source at distance 8.18.


YTFieldNotFound: Could not find field ('gas', 'temperature') in UniformGridData.

In [ ]:
L = [ 0.91354546, 0.15236623,  -0.37711965]
North_vector = [3.74606593e-01,  -9.27183855e-01, 3.74606593e-01]
n_events = pyxsim.project_photons("my_photonstestMHD15m2Ms", "my_photonstestMHD_REVabs_dp15m2Ms",L ,(266.42, -29),nH = 11,
                                  absorb_model = "tbabs", no_shifting = False, north_vector = North_vector, column_file = "internal_abs_REV.h5")

In [ ]:
events = pyxsim.EventList("no_abs_my_photonstestMHD_REVabs_dp15m2Ms.h5")
events.write_to_simput("No_REVabsorption_MHD_dp15m2Ms", overwrite = True)